In [ ]:
!pip install -q asn1crypto==1.4.0
!pip install -q certifi==2021.10.8
!pip install -q cffi==1.15.0
!pip install -q charset-normalizer==2.0.9
!pip install -q cryptography==35.0.0
!pip install -q dataclasses==0.6
!pip install -q idna==3.3
!pip install -q oscrypto==1.2.1
!pip install -q pycparser==2.21
!pip install -q pycryptodomex==3.12.0
!pip install -q PyJWT==2.3.0
!pip install -q pyOpenSSL==21.0.0
!pip install -q pytz==2021.3
!pip install -q requests==2.26.0
!pip install -q six==1.16.0
!pip install -q urllib3==1.26.7
!pip install -q snowflake-connector-python==2.7.2
!pip install -q "snowflake-connector-python[pandas]"
!pip install -q geopandas

!export TMPDIR=/large_tmp_volume
# %env TMPDIR=/drive/My Drive/Personal/Snowflake

In [17]:
import pandas as pd
import json
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon
from folium import Map, Marker, GeoJson
import branca.colormap as cm
import geojson
from geojson import Feature, FeatureCollection

from shapely.geometry import shape

import snowflake.connector
# %env TMPDIR=/drive/My Drive/Personal/Snowflake

!export TMPDIR=/large_tmp_volume


In [18]:
f = open('credentials.json')
data = json.load(f)
f.close()

conn = snowflake.connector.connect(
    user=data["user"],
    password=data["password"],
    account=data["account"]
    )

In [19]:
cur = conn.cursor()
cur.execute("use schema OSM_NEWYORK.NEW_YORK")
cur.execute("alter session set geography_output_format = 'GEOJSON'")
cur.execute("select * from V_OSM_NY_AMENITY_TRANSPORTATION where TYPE = 'way' and capacity = capacity")
df = cur.fetch_pandas_all()

In [20]:
df['COORDINATES'] = df['COORDINATES'].apply(lambda x: geojson.loads(x))

In [21]:
df = df[df['CAPACITY'].isin(['about 30 cars', '3 cars', '200+','>200','apx. +200',
                             '1,500','1,850','>100','1,659']) == False]
df["CAPACITY"] = pd.to_numeric(df["CAPACITY"].copy())

In [22]:
df['geometry'] = df['COORDINATES']

In [27]:
def dataframe_to_geojson(df_geo, column_name, file_output = None):
    
    '''Produce the GeoJSON for a dataframe that has a geometry column in geojson format already'''
    
    list_features = []
    
    for i,row in df_geo.iterrows():
        feature = Feature(geometry = row["geometry"], properties = {column_name : row[column_name]})
        list_features.append(feature)
        
    feat_collection = FeatureCollection(list_features)
    
    geojson_result = json.dumps(feat_collection)
    
    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)
    
    return geojson_result


def choropleth_map(df_aggreg, column_name, border_color = 'none', fill_opacity = 0.5, initial_map = None, with_legend = True,
                   kind = "linear"):
    #colormap
    min_value = df_aggreg[column_name].min()
    max_value = df_aggreg[column_name].max()
    m = round ((min_value + max_value ) / 2 , 0)
    
    
    if initial_map is None:
        initial_map = Map(location= [43.083658, -76.221406], zoom_start=14, tiles="cartodbpositron")

    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(['gray','blue','green','yellow','orange','red'], vmin=min_value, vmax=max_value)
    elif kind == "outlier":
        #for outliers, values would be -11,0,1
        custom_cm = cm.LinearColormap(['white','gray','black'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['sienna','green','yellow','red'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)
   
    #create geojson data from dataframe
    geojson_data = dataframe_to_geojson(df_geo = df_aggreg, column_name = column_name)
    
    #plot on map
    name_layer = "Choropleth "
    if kind != "linear":
        name_layer = name_layer + kind
        
    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties'][column_name]),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)

    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)
        
    return initial_map

In [28]:
map = choropleth_map(df_aggreg = df[df['CAPACITY'] > 100], column_name = "CAPACITY")
map